# Step 1

## Extract highways & nodes using Osmium
## Import & integrate raw Mapbox data 

This script is the first step in the mapbox workflow. Note that this works outside of the existing gostnets methodology in order to preserve all nodes and not simplify data and remove osm_ids. The current gn methodology does not use osm_ids, but these are needed to join mapbox data to osm. developed by Tom Gertin. 


In [2]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append(r"C:\Users\wb557957\Desktop\GOSTnets-master")
import GOSTnets as gn

# pip install osmium
import osmium, logging
import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

import time
import psutil
import tracemalloc

In [3]:
pth = r"C:\Users\wb557957\Desktop\LKA"
inPth = r"C:\Users\wb557957\Desktop\LKA\lka_data"
outPth = r"C:\Users\wb557957\Desktop\LKA\lka_outputs"

#kandy_osm = r'C:\Users\wb557957\Desktop\LKA\lka_data\kandy_roads.osm.pbf'

In [26]:
# instead of exporting roads specific to kandy, you can also use gpd.clip on the existing graph.
# I had AttributeError geopandas module has no attribute clip, so i just re-ran the whole model.
# C = nx.read_gpickle(os.path.join(pth, r'C_combinedGraphs.pickle')) 

This script will run the Sri Lanka methodology in the Kandy (Central) region. 

The 87 tourism destinations used were extracted from a 1 hour service area from Kandy city. Service areas were calculated using OSM roads and the service areas tools in ArcGIS.

## Step 1: Extracting highways and nodes using Osmium

In [9]:
'''start_time = time.time()

wkbfab = osmium.geom.WKBFactory()

# extract highways
class HighwayExtractor(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self) 
        self.nodes = []
        #self.raw_h = []
        self.highways = []
        self.broken_highways = []
        self.total = 0
        self.num_nodes = 0
    
    def way(self, w):
        #self.raw_h.append(w)
        try:
            nodes = [x.ref for x in w.nodes]
            wkb = wkbfab.create_linestring(w)
            shp = wkblib.loads(wkb, hex=True)
            if 'highway' in w.tags:
                info = [w.id, nodes, shp, w.tags['highway']]
                self.highways.append(info)
        except:
            print('hit exception')
            nodes = [x for x in w.nodes if x.location.valid()]
            if len(nodes) > 1:
                shp = LineString([Point(x.location.x, x.location.y) for x in nodes])
                info = [w.id, nodes, shp, w.tags['highway']]
                self.highways.append(info)
            else:
                self.broken_highways.append(w)
            logging.warning("Error Processing OSM Way %s" % w.id)
            
        
h = HighwayExtractor()
h.apply_file(kandy_osm, locations=True)
print(len(h.highways))
print(len(h.broken_highways))

end_time = time.time()
print('time (in min)')
print((end_time - start_time)*60)'''

35900
0
time (in min)
4084.349112510681


In [10]:
#h.highways[1]

[6105649,
 [5764078202,
  3464794530,
  5764078201,
  3464794537,
  5764078200,
  3224464663,
  3464794548,
  3464794557,
  3464794595,
  5764078199,
  50428831,
  3224464665,
  5764078197,
  3224464664,
  5764078196,
  5764078198,
  1140291133,
  5764078195,
  1140291372,
  5764078194,
  3464794562,
  5764078193,
  1030050506,
  5764078192,
  3224464667,
  3224464666,
  1140291621,
  3464794607,
  3224464668,
  5764078191,
  1030050721,
  5764078190,
  3464794533,
  3464794578,
  5764078189,
  1140291043,
  5764078188,
  3724532648,
  3224464669,
  5764078187,
  1030050906,
  3724512192,
  3464794563,
  3224464670,
  3224464671,
  1030050524,
  5764078186,
  3464794561,
  3224464672,
  3464794613,
  3224464673,
  5764078185,
  1030050743,
  3464794591,
  3224464675,
  3224464674,
  3464794539,
  3224464676,
  1140291308,
  3464794615,
  3464794611,
  3224464677,
  5764078184,
  5764078179,
  1140291562,
  3224464678,
  3224464680,
  1030050918,
  3464794585,
  3224464679,
  5764078183

In [11]:
#h.highways[2][2].length

0.001153960101113727

In [12]:
#list(h.highways[0][2].coords)

[(80.6333162, 7.2931692),
 (80.6331674, 7.2930943),
 (80.633093, 7.2930606),
 (80.6330609, 7.2930489),
 (80.6328746, 7.2929548),
 (80.6326017, 7.2928072),
 (80.6325315, 7.2927622),
 (80.6324507, 7.2927038),
 (80.6323806, 7.292649),
 (80.6322647, 7.2925403),
 (80.6321154, 7.2923985),
 (80.6320988, 7.2923801),
 (80.6318931, 7.2920987),
 (80.6318577, 7.2920503),
 (80.631747, 7.2918988),
 (80.6316457, 7.2917602),
 (80.6313858, 7.2914047),
 (80.631295, 7.2912804),
 (80.6312477, 7.2912157),
 (80.631199, 7.2911505),
 (80.6310755, 7.2909851),
 (80.6306367, 7.2903975),
 (80.630412, 7.2900967),
 (80.6303381, 7.2900085),
 (80.6300817, 7.2897028),
 (80.6299062, 7.289482),
 (80.6298466, 7.289407),
 (80.6297885, 7.2893339),
 (80.6297468, 7.289284),
 (80.6295988, 7.2891013),
 (80.6294034, 7.28888),
 (80.6292219, 7.2886911),
 (80.6290277, 7.2885073),
 (80.628942, 7.2884223),
 (80.6288638, 7.2883247),
 (80.6287239, 7.2881146),
 (80.6286041, 7.2879583),
 (80.6284932, 7.287832),
 (80.6284268, 7.2877642),

In [13]:
#list(h.highways[2][2].coords)[2]

(80.7588193, 6.9823604)

### collect osm ID's of all highways

In [14]:
start_time = time.time()

all_nodes = []
all_edges = []

for x in h.highways:
    for n_idx in range(0, (len(x[1]) - 1)):
        try:
            osm_id_from = x[1][n_idx].ref
        except:
            osm_id_from = x[1][n_idx]
        try:
            osm_id_to   = x[1][n_idx+1].ref
        except:
            osm_id_to   = x[1][n_idx+1]
        try:
            osm_coords_from = list(x[2].coords)[n_idx]
            #print(osm_coords_from[0])
            #create a node
            all_nodes.append([osm_id_from, { 'x' : osm_coords_from[0], 'y' : osm_coords_from[1] }])
            osm_coords_to = list(x[2].coords)[n_idx+1]
            #print(n_idx)
            #print(len(x[1]) - 1)
            if n_idx == (len(x[1]) - 2):
                #print('last element')
                #print(osm_coords_to)
                #create a node
                all_nodes.append([osm_id_to, { 'x' : osm_coords_to[0], 'y' : osm_coords_to[1]} ])
            edge = LineString([osm_coords_from, osm_coords_to])
            attr = {'osm_id':x[0], 'Wkt':edge, 'length':edge.length, 'infra_type':x[3]}
            #Create an edge from the list of nodes in both directions
            all_edges.append([osm_id_from, osm_id_to, attr])
            all_edges.append([osm_id_to, osm_id_from, attr])
        except:
            logging.warning(f"Error adding edge between nodes {osm_id_from} and {osm_id_to}")

end_time = time.time()
print(end_time - start_time)

143.24378991127014


In [15]:
all_nodes[:10]

[[35436481, {'x': 80.6333162, 'y': 7.2931692}],
 [2078304672, {'x': 80.6331674, 'y': 7.2930943}],
 [3810950074, {'x': 80.633093, 'y': 7.2930606}],
 [5504401132, {'x': 80.6330609, 'y': 7.2930489}],
 [3057458327, {'x': 80.6328746, 'y': 7.2929548}],
 [2266684575, {'x': 80.6326017, 'y': 7.2928072}],
 [4004621091, {'x': 80.6325315, 'y': 7.2927622}],
 [3991952783, {'x': 80.6324507, 'y': 7.2927038}],
 [3780934139, {'x': 80.6323806, 'y': 7.292649}],
 [35436482, {'x': 80.6322647, 'y': 7.2925403}]]

In [16]:
all_edges[:10]

[[35436481,
  2078304672,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestring.LineString at 0x1f40abe4b70>,
   'length': 0.00016658766460147465,
   'infra_type': 'trunk'}],
 [2078304672,
  35436481,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestring.LineString at 0x1f40abe4b70>,
   'length': 0.00016658766460147465,
   'infra_type': 'trunk'}],
 [2078304672,
  3810950074,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestring.LineString at 0x1f40abe4be0>,
   'length': 8.167649600961457e-05,
   'infra_type': 'trunk'}],
 [3810950074,
  2078304672,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestring.LineString at 0x1f40abe4be0>,
   'length': 8.167649600961457e-05,
   'infra_type': 'trunk'}],
 [3810950074,
  5504401132,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestring.LineString at 0x1f40abe4c18>,
   'length': 3.4165772344625784e-05,
   'infra_type': 'trunk'}],
 [5504401132,
  3810950074,
  {'osm_id': 5131121,
   'Wkt': <shapely.geometry.linestri

In [21]:
# create graph object with nodes and edges from the above.
G = nx.MultiDiGraph()
G.add_nodes_from(all_nodes)
G.add_edges_from(all_edges)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [18]:
gn.save(G, 'G_kandy', './', pickle=True, nodes=False, edges=True)

In [19]:
# create a speed dictionary to calculate speed-based travel time values for each nx edge
# all speeds are in kmph
speed_dict = {
                'primary': 60,
                'primary_link': 50,
                'secondary': 50,
                'secondary_link': 45,
                'tertiary':40,
                'tertiary_link': 40,
                'residential': 30,
                'trunk': 60,
                'trunk_link': 50,
                'unclassified': 25,
                'track': 25,
                'service': 20
                }

In [22]:
G_time_kandy = gn.convert_network_to_time(G, 
                                    distance_tag = 'length', 
                                    road_col = 'infra_type', 
                                    speed_dict = speed_dict, 
                                    factor = 1000)

In [23]:
gn.save(G_time_kandy, 'G_time_kandy', './', pickle=True, nodes=False, edges=True) 

In [24]:
edges_gdf = gn.edge_gdf_from_graph(G_time_kandy)
edges_gdf.to_csv('G_time_kandy_edges.csv')

In [25]:
nodes_gdf = gn.node_gdf_from_graph(G_time_kandy)
nodes_gdf.to_csv('G_time_kandy_nodes.csv')

## Step 2: Concatenate & simplify mapbox data

In [26]:
# read in each of the three traffic .csvs
traffic_csv1 = os.path.join(inPth, "1233300-Asia-Colombo.csv")
traffic_csv2 = os.path.join(inPth, "1233302-Asia-Colombo.csv")
traffic_csv3 = os.path.join(inPth, "1233303-Asia-Colombo.csv")

In [27]:
traffic1 = pd.read_csv(traffic_csv1, header = None)
traffic1.head()

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40


In [28]:
traffic2 = pd.read_csv(traffic_csv2, header = None)
traffic2.head()

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,102738212,3279663552,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
1,1093698450,1093697790,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
2,1093698671,1093697934,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
3,1093798472,1093799200,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
4,1093798791,1093799519,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34


In [29]:
traffic3 = pd.read_csv(traffic_csv3, header = None)
traffic3.head()

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1147027495,3318178916,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
1,1147029008,3333484939,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
2,1147030124,3318163467,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,1675585654,1675585568,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
4,1675667479,5787866342,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43


In [30]:
# join traffic_simplified, traffic_simplified2, traffic_simplified3
traffic_3df = [traffic1, traffic2, traffic3]

traffic_all = pd.concat(traffic_3df)
traffic_all.head()

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40


In [31]:
traffic_all.count()

0       502040
1       502040
2       502040
3       502040
4       502040
         ...  
2013    502040
2014    502040
2015    502040
2016    502040
2017    502040
Length: 2018, dtype: int64

#### now, calculate statistics on speed values and simplify the fields

In [36]:
import scipy
def get_speeds(x):
    # x_vals is the value of all columns in traffic data from 2 onward
    x_vals = x[2:]
    return([min(x_vals), max(x_vals), np.mean(x_vals), scipy.stats.mode(x_vals), (np.mean(x_vals) - min(x_vals))]) #, np.argmax(x_vals)

traffic_vals = traffic_all.apply(lambda x: get_speeds(x), axis = 1, result_type = "expand")
traffic_vals.columns = ['min_speed','max_speed','mean_speed','mode_speed', 'diffMeanMin']

In [37]:
traffic_vals

,min_speed,max_speed,mean_speed,mode_speed,diffMeanMin
0,31,38,36.866071,"([37], [1866])",5.866071
1,57,57,57.000000,"([57], [2016])",0.000000
2,60,60,60.000000,"([60], [2016])",0.000000
3,46,46,46.000000,"([46], [2016])",0.000000
4,40,40,40.000000,"([40], [2016])",0.000000
...,...,...,...,...,...
9921,36,60,46.133929,"([46], [1876])",10.133929
9922,20,26,25.888393,"([26], [1951])",5.888393
9923,10,11,10.977679,"([11], [1971])",0.977679
9924,64,70,66.014881,"([66], [1941])",2.014881


In [38]:
# insert headers for from and to node columns and join to the speeds df
traffic_simplified = traffic_all.loc[:,[0,1]]
traffic_simplified.columns = ['FROM_NODE', "TO_NODE"]
traffic_simplified = traffic_simplified.join(traffic_vals)

In [39]:
print(traffic_simplified.count())
# total unique edges w traffic: 502,040
# doesn't include duplicate nodes

FROM_NODE      590246
TO_NODE        590246
min_speed      590246
max_speed      590246
mean_speed     590246
mode_speed     590246
diffMeanMin    590246
dtype: int64


In [40]:
print(traffic_simplified.head())

    FROM_NODE     TO_NODE  min_speed  max_speed  mean_speed      mode_speed  \
0  1148494884  4177608798         31         38   36.866071  ([37], [1866])   
0  1148494884  4177608798         34         57   45.918651   ([49], [817])   
0  1148494884  4177608798         58         79   66.348710   ([71], [874])   
0   102738212  3279663552         31         38   36.866071  ([37], [1866])   
0   102738212  3279663552         34         57   45.918651   ([49], [817])   

   diffMeanMin  
0     5.866071  
0    11.918651  
0     8.348710  
0     5.866071  
0    11.918651  


#### filter the attributed_edges and the edges_gdf for only road segments that interest us.
having trouble with it. not sure why

In [48]:
attributed_edges.infra_type.value_counts()

trunk             67556
secondary         52195
primary           26779
tertiary          19428
residential        9931
unclassified       4021
service             799
living_street       267
trunk_link           82
primary_link         20
motorway_link        19
secondary_link       19
tertiary_link         7
Name: infra_type, dtype: int64

In [49]:
edges_gdf.infra_type.value_counts()

residential       752576
unclassified      441380
tertiary          286818
track             281926
secondary         162694
service            93738
trunk              76298
primary            42256
path               36782
living_street      35466
footway            10750
road                5290
construction        1394
steps                742
trunk_link           332
tertiary_link        204
secondary_link        88
primary_link          76
pedestrian            72
raceway               60
motorway_link         38
bridleway             26
cycleway              22
services              12
Name: infra_type, dtype: int64

In [30]:
import GOSTnets.load_osm as losm
def filterRoads(self, acceptedRoads = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link', 'motorway', 'motorway_link']):
        """
        Extract certain times of roads from the OSM before the netowrkX conversion 

        :param acceptedRoads: [ optional ] acceptedRoads [ list of strings ] 
        :returns: None - the raw roads are filtered based on the list of accepted roads
        """

        self = self.loc[self.infra_type.isin(acceptedRoads)]

In [31]:
mb_edges_filter = filterRoads(attributed_edges, acceptedRoads = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link', 'motorway', 'motorway_link'])
# all_edges_filter = filterRoads(edge_gdf, acceptedRoads = accepted_road_types)

In [36]:
print(mb_edges_filter)

None


## Step 3: Combine Mapbox and OSM data into one Graph
the previous operation calculated statistics and speed values on the entire country. we now want to merge mapbox data into the kandy edges.

first we will add attributes to mapbox data so that the two gdfs can be joined. the subsequent step will be to perform the merge between 'attributed_edges' and the full osm roads gdf.

### * note:  this methodology didn't quite work, so I brought the csv into GIS and 1. combined speeds, 2. reprojected to UTM 44N, 3. calculated travel time.

In [ ]:
# re-import graph and edges if they weren't opened in this session
G_kandy = nx.read_gpickle(os.path.join(pth, r'G_time_kandy.pickle'))
# kandy_edges = pd.read_csv('G_time_kandy_edges.csv')

In [44]:
# locate the node in the osm edges gdf and the from node in the traffic gdf.
edges_gdf.loc[edges_gdf['stnode'] == 4965331097]
traffic_simplified.loc[traffic_simplified['FROM_NODE'] == 4965331097]

,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed,mode_speed,diffMeanMin


In [45]:
# merge the traffic speeds to the edges where they match.
attributed_edges = edges_gdf.merge(traffic_simplified, left_on=['stnode','endnode'], right_on=['FROM_NODE','TO_NODE'])
attributed_edges

,stnode,endnode,Wkt,infra_type,osm_id,mode,time,length,geometry,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed,mode_speed,diffMeanMin
0,35436481,2078304672,"LINESTRING (80.6333162 7.2931692, 80.6331674 7...",trunk,5131121,drive,0.009995,0.166588,"LINESTRING (80.63332 7.29317, 80.63317 7.29309)",35436481,2078304672,5,15,6.029762,"([6], [1976])",1.029762
1,35436481,3780934137,"LINESTRING (80.6333162 7.2931692, 80.633701900...",trunk,23802729,drive,0.025510,0.425170,"LINESTRING (80.63332 7.29317, 80.63370 7.29335)",35436481,3780934137,1,3,1.044643,"([1], [1971])",0.044643
2,35436481,4004621090,"LINESTRING (80.6333162 7.2931692, 80.6333353 7...",secondary,23802869,drive,0.006132,0.085169,"LINESTRING (80.63332 7.29317, 80.63334 7.29325)",35436481,4004621090,7,10,7.049603,"([7], [1971])",0.049603
3,2078304672,35436481,"LINESTRING (80.6333162 7.2931692, 80.6331674 7...",trunk,5131121,drive,0.009995,0.166588,"LINESTRING (80.63317 7.29309, 80.63332 7.29317)",2078304672,35436481,1,15,1.128968,"([1], [1976])",0.128968
4,2078304672,3810950074,"LINESTRING (80.6331674 7.2930943, 80.633093 7....",trunk,5131121,drive,0.004901,0.081676,"LINESTRING (80.63317 7.29309, 80.63309 7.29306)",2078304672,3810950074,6,10,7.983135,"([8], [1939])",1.983135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181118,7227878070,7227878058,"LINESTRING (80.63723659999999 7.3090689, 80.63...",trunk,774684969,drive,0.002367,0.039444,"LINESTRING (80.63724 7.30907, 80.63723 7.30911)",7227878070,7227878058,17,17,17.000000,"([17], [2016])",0.000000
181119,7227878058,7227878059,"LINESTRING (80.6372295 7.3091077, 80.637212199...",trunk,774684969,drive,0.002748,0.045794,"LINESTRING (80.63723 7.30911, 80.63721 7.30915)",7227878058,7227878059,17,17,17.000000,"([17], [2016])",0.000000
181120,7227878059,7227878071,"LINESTRING (80.63721219999999 7.3091501, 80.63...",trunk,774684969,drive,0.002235,0.037252,"LINESTRING (80.63721 7.30915, 80.63719 7.30918)",7227878059,7227878071,17,17,17.000000,"([17], [2016])",0.000000
181121,7227878071,7227878060,"LINESTRING (80.6371911 7.3091808, 80.6371698 7...",trunk,774684969,drive,0.001602,0.026700,"LINESTRING (80.63719 7.30918, 80.63717 7.30920)",7227878071,7227878060,17,17,17.000000,"([17], [2016])",0.000000


In [47]:
attributed_edges.count()
# total edges in kandy with mapbox: 181,123

stnode         181123
endnode        181123
Wkt            181123
infra_type     181123
osm_id         181123
mode           181123
time           181123
length         181123
geometry       181123
FROM_NODE      181123
TO_NODE        181123
min_speed      181123
max_speed      181123
mean_speed     181123
mode_speed     181123
diffMeanMin    181123
dtype: int64

In [46]:
attributed_edges.to_csv('lka_MB_osmAttributes.csv')

## new kernel? start here

In [4]:
#attributed mapbox edges in kandy only
attributed_edges = pd.read_csv('lka_MB_osmAttributes.csv')

In [5]:
G_time_kandy = nx.read_gpickle('G_time_kandy.pickle')

In [6]:
# For a nx.MultiGraph you need an extra index going from 0 to n where n is the number of edges between the two nodes.

# here we manually update the mapbox traffic speed with the mean value, as a test.
G_time_kandy[35436481][2078304672][0]['mapbox_traffic_speed'] = 6.029762

In [7]:
G_time_kandy[35436481][2078304672]

AtlasView({0: {'osm_id': 5131121, 'Wkt': <shapely.geometry.linestring.LineString object at 0x0000029616EC8C50>, 'length': 0.16658766460147464, 'infra_type': 'trunk', 'time': 0.009995259876088479, 'mode': 'drive', 'mapbox_traffic_speed': 6.029762}})

In [8]:
G_time_kandy[2078304672][3810950074]

AtlasView({0: {'osm_id': 5131121, 'Wkt': <shapely.geometry.linestring.LineString object at 0x0000029616EC8E80>, 'length': 0.08167649600961457, 'infra_type': 'trunk', 'time': 0.004900589760576874, 'mode': 'drive'}})

In [9]:
def add_edge_attribute(x):
    #print(x['osm_id'])
    #print(int(x['mean_speed']))
    G_time_kandy[x['stnode']][x['endnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    G_time_kandy[x['endnode']][x['stnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    
attributed_edges.apply(lambda x: add_edge_attribute(x), axis = 1)

0         None
1         None
2         None
3         None
4         None
          ... 
181118    None
181119    None
181120    None
181121    None
181122    None
Length: 181123, dtype: object

In [ ]:
# try adding mean_speed and nominal speed from G_time.
# note that 'time' is a value from the nominal speeds, not from mean traffic speed. 
G_time_kandy.edges

In [11]:
G_time_kandy[35436481][2078304672]

AtlasView({0: {'osm_id': 5131121, 'Wkt': <shapely.geometry.linestring.LineString object at 0x0000029616EC8C50>, 'length': 0.16658766460147464, 'infra_type': 'trunk', 'time': 0.009995259876088479, 'mode': 'drive', 'mapbox_traffic_speed': 1}})

In [12]:
nx.number_of_edges(G_time_kandy)

2229040

In [38]:
kandy_edge_w_traffic = gn.edge_gdf_from_graph(G_time_kandy)

In [39]:
kandy_edge_w_traffic.head()

,stnode,endnode,time,mode,infra_type,osm_id,mapbox_traffic_speed,length,Wkt,geometry
0,35436481,2078304672,0.009995,drive,trunk,5131121,1.0,0.166588,"LINESTRING (80.6333162 7.2931692, 80.6331674 7...","LINESTRING (80.63332 7.29317, 80.63317 7.29309)"
1,35436481,3780934137,0.025510,drive,trunk,23802729,4.0,0.425170,"LINESTRING (80.6333162 7.2931692, 80.633701900...","LINESTRING (80.63332 7.29317, 80.63370 7.29335)"
2,35436481,4004621090,0.006132,drive,secondary,23802869,8.0,0.085169,"LINESTRING (80.6333162 7.2931692, 80.6333353 7...","LINESTRING (80.63332 7.29317, 80.63334 7.29325)"
3,2078304672,35436481,0.009995,drive,trunk,5131121,1.0,0.166588,"LINESTRING (80.6333162 7.2931692, 80.6331674 7...","LINESTRING (80.63317 7.29309, 80.63332 7.29317)"
4,2078304672,3810950074,0.004901,drive,trunk,5131121,4.0,0.081676,"LINESTRING (80.6331674 7.2930943, 80.633093 7....","LINESTRING (80.63317 7.29309, 80.63309 7.29306)"


### Export edges as csv and shapefile

In [40]:
# simplify the table before exporting
kandy_edge_w_traffic_export = kandy_edge_w_traffic[['stnode','endnode','osm_id','length','infra_type','mapbox_traffic_speed','geometry']]

In [41]:
kandy_edge_w_traffic_export.to_csv('kandy_edge_w_traffic.csv')

In [ ]:
# kandy_edge_w_traffic_export.to_file(driver = 'ESRI Shapefile', filename = 'kandy_edge_w_traffic.shp')

In [42]:
# if export to .shp doesnt work, try exporting to geojson. 
kandy_edge_w_traffic_export.to_file(driver = 'GeoJSON', filename = 'kandy_edge_w_traffic.json')